# Vacancy Formation Energy of Copper

<b>Task</b>: to find the vacancy formation energy in bulk copper

The vacancy formation energy is the energy required to break the bonds between an atom inside the crystal and its nearest neighbor atoms and removing that atom to where it has no interaction with the remaining system. [1]

The vacancy formation energy can be found by the equation: <center>$E_v^f = E_f - N_v \cdot E_{ref}$ </center> where $E_f$, $N_v$ and $E_{ref}$ are the energy of the relaxed system with vacancy, the number of atoms in the system with vacancy and energy per unit volume of the material respectively.

The energy of the cluster is calculated using Morse Potential with parameters from [2]. The Morse potential between atoms $i$ and $j$ is <center> $V_{ij} = D\big( e^ { -2 \alpha (r_{ij} - r_e) } - 2e^{ - \alpha (r_{ij} - r_e)  } \big) $ </center> where $D$, $\alpha$, $r_{ij}$ and $r_e$ are the well depth, the width of the potential, the equilibrium atomic distance.

A table showing how the vacancy formation energy changes with cluster size is generated at the end.

<b>References</b>: 

[1]https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_Vacancy_Formation_Energy

[2]Oluwajobi, A. & Chen, X. Int. J. Autom. Comput. (2011) 8: 326. https://doi.org/10.1007/s11633-011-0588-y

In [1]:
using JuLIP, Optim, DataFrames

In [2]:
# Morse potential
D = 0.3429 # well depth
alpha = 1.3588 # width of potential (Argstrom)
r_e = 2.866 # atomic distance at equilibrium
cut_off = 6.4 # cut_off radius
mp = (@analytic r -> D * ( exp(-2*alpha*(r-r_e)) - 2*exp(-alpha*(r-r_e)) )) * SplineCutoff(0.01, cut_off)

JuLIP.Potentials.AnalyticFunction{##1#4,##2#5,##3#6}(#1, #2, #3) * JuLIP.Potentials.SplineCutoff(0.01, 6.4)

In [3]:
# reference energy
# energy per unit volume of a homogeneous copper crystal
at = bulk(:Cu)
set_calculator!(at, mp)
Eref = energy(at)

-1.426054040194669

In [4]:
function VFE(size)
    # cluster with vacancy
    at = bulk(:Cu, cubic = true) * size
    set_calculator!(at, mp)
    deleteat!(at, length(at) ÷ 2)
    # energy before relaxing
    E0 = energy(at) - length(at)*Eref

    # geometry optimization
    set_constraint!(at, FixedCell(at))
    result = minimise!(at)
    E1 = result.minimum - length(at)*Eref
    
    return E0, E1
end

VFE (generic function with 1 method)

In [ ]:
size_range = 3:10
VFE_0, VFE_1 = [ VFE(n)[1] for n in size_range ], [ VFE(n)[2] for n in size_range ]  

In [6]:
df = DataFrame(Size = size_range, Unrelaxd_VFE = VFE_0, Relaxed_VFE = VFE_1)

,Size,Unrelaxd_VFE,Relaxed_VFE
1,3,1.42605,1.28393
2,4,1.42605,1.27998
3,5,1.42605,1.27881
4,6,1.42605,1.27835
5,7,1.42605,1.27813
6,8,1.42605,1.27801
7,9,1.42605,1.27794
8,10,1.42605,1.2779
